# Import Config


In [6]:
# Load the YAML file
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/VICReg-BarlowTwins-Ablation-Study/autoSSL")
sys.path.append("C:/Users/isxzl/OneDrive/Code/VICReg-BarlowTwins-Ablation-Study/")

import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer
import pytorch_lightning as pl

with open('../experiment/global.yaml', 'r') as file:
    config = yaml.safe_load(file)


# Define global view function
SSL_augmentation = {
    'RandomResizedCrop': {'size': (config["input_size"], config["input_size"])},
    'RandomApply':{'transforms':[RandomRotation(degrees=90)], 'p':0.8},
    'RandomHorizontalFlip': {'p': 0.5},
    'RandomVerticalFlip':  {'p':0.5},
    'RandomApply':{'transforms': [ColorJitter(brightness=0.04,contrast=0.04,saturation=0.02,hue=0.01)], 'p':0.8},
    'RandomGrayscale' :{'p':0.2},
    'RandomSolarize':{'threshold':128, 'p':0.1},
    'RandomApply':{'transforms':[GaussianBlur(kernel_size=3,sigma=(0.2, 2))],'p':0.8},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}



# Import the dataset

In [10]:
# THIS IS THE CODE TO LOAD DATASET

pdata= PipeDataset(input_dir=config["dataset_dir"], 
      augmentation=dict2transformer(SSL_augmentation,view=config["view"]), 
      samples=config["samples"], batch_size=config["batch_size"], shuffle=config["shuffle"], 
      drop_last=config["drop_last"], num_workers=config["num_workers"] 
    ) 
        
        

# Create a Model!!

In [11]:

pmodel=pipe_model(name=config["model"], backbone=config["backbone"],
           stop_gradient=config["stop_gradient"], prjhead_dim=config["prjhead_dim"])


# Training the data!

In [12]:
trainer = pl.Trainer(
    max_epochs=config["max_epochs"],accelerator=config["device"], 
    #callbacks=[ck_callback(config["log_dir"])],
    #logger=ContinuousCSVLogger(save_dir=config["log_dir"],name=config["name"])
)


#for train
trainer.fit(model, pdata.dataloader)


# FOR LOAD PREVIOUS WEIGHT
#trainer.fit(model, pdata.dataloader,ckpt_path=(join_dir(config["log_dir"],"checkpoints-epoch=02-train_loss=20.89.ckpt")))  



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: C:\Users\isxzl\OneDrive\Code\VICReg-BarlowTwins-Ablation-Study\Tutorial_for_library\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                      | Params
--------------------------------------------------------------
0 | backbone        | Sequential                | 11.2 M
1 | projection_head | BarlowTwinsProjectionHead | 9.4 M 
2 | criterion       | VICRegLoss                | 0     
----------------------------

Epoch 0:  50%|████████████████████████████████                                | 49/98 [00:23<00:23,  2.10it/s, v_num=0]

C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\trainer\call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


 # Evaluate

In [13]:
# Define global view function
test_augmentation = {
    'RandomResizedCrop': {'size': (64, 64)},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}

pipe_data_test =PipeDataset(
    input_dir="../../Datasets/cifar10/test/", 
    samples=1000, 
    augmentation=dict2transformer(test_augmentation), 
    batch_size=32,
    shuffle=True,
    #collate_fn=collate_fn,
    drop_last=True,
    num_workers=4,

) 

eval_linear(pipe_data_test,models=model,split=0.8,device="cuda") 
#eval_linear(pipe_data_test,models=model,test=pipe_data_test,device="cuda") 
        


Load the training and testing dataset



100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  3.74it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  3.79it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:06<00:00,  1.11s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:06<00:00,  1.11s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.94it/s]


[('model_0', 0.09375)]